In [126]:
import cobra
from cobra.flux_analysis import production_envelope, pfba, flux_variability_analysis, moma
from cobra.flux_analysis.loopless import add_loopless, loopless_solution
from cobra.medium import minimal_medium
from cobra.sampling import sample
from optlang import gurobi_interface

import pandas as pd
# from plotnine import ggplot, geom_point, aes, stat_smooth, facet_wrap

In [149]:
# import model

# model_path = '../../models/json/iML1515.json'
# model_path = '../../models/json/GM_iML1515.json'
model_path = '../../models/json/GMglud_iML1515.json'
# model_path = '../../models/json/iJO1366.json'
model = cobra.io.load_json_model(model_path)

# model.solver = 'glpk'
# model.solver = 'gurobi'

In [64]:
# test atp generation

with model:

    model.objective = 'ATPM'
    
    medium = model.medium
    # medium.update({'EX_glc__D_e': 1.0, 'EX_o2_e': 20.0})
    medium.update({'EX_glc__D_e': 1.0, 'EX_phleth_e': 0.0, 'EX_o2_e': 20.0})
    model.medium = medium
    
    model.reactions.get_by_id('ATPM').upper_bound = 1000.0
    
    solution = model.optimize()
    # solution = pfba(model)
    # print(dir(solution))
    # print(solution.reduced_costs)

    print('atp yield: ', solution.objective_value);
    print(model.metabolites.atp_c.summary());

    # from 1 glucose: 
    # 2 atp from pgk (glycolysis)
    # 2 atp from sucoas (tca cycle)
    # 19.5 atp from atps4rpp (atp synthase within periplasm)
    # total 23.5 atp produced

atp yield:  23.49999999999999
atp_c
=====
Formula: C10H12N5O13P3

Producing Reactions
-------------------
Percent Flux Reaction                                         Definition
 82.98% 19.5 ATPS4rpp adp_c + 4.0 h_p + pi_c <=> atp_c + h2o_c + 3.0 h_c
  8.51%    2      PGK                  3pg_c + atp_c <=> 13dpg_c + adp_c
  8.51%    2   SUCOAS atp_c + coa_c + succ_c <=> adp_c + pi_c + succoa_c

Consuming Reactions
-------------------
Percent  Flux Reaction                           Definition
100.00% -23.5     ATPM atp_c + h2o_c --> adp_c + h_c + pi_c


In [105]:
# wild type yield

with model:
    medium = model.medium
    medium.update({'EX_glc__D_e': 10.0, 'EX_o2_e': 20.0})
    # medium.update({'EX_glc__D_e': 10.0, 'EX_phleth_e': 0.0, 'EX_o2_e': 20.0})
    # medium.update({'EX_pi_e': 1000.0, 'EX_co2_e': 0, 'EX_fe3_e': 1000.0, 'EX_h_e': 1000.0, 'EX_mn2_e': 1000.0, 'EX_fe2_e': 0, 'EX_glc__D_e': 10.0, 'EX_zn2_e': 1000.0, 'EX_mg2_e': 1000.0, 'EX_ca2_e': 1000.0, 'EX_ni2_e': 0, 'EX_cu2_e': 1000.0, 'EX_sel_e': 0, 'EX_cobalt2_e': 1000.0, 'EX_h2o_e': 1000.0, 'EX_mobd_e': 1000.0, 'EX_so4_e': 1000.0, 'EX_nh4_e': 1000.0, 'EX_k_e': 1000.0, 'EX_na1_e': 1000.0, 'EX_cl_e': 1000.0, 'EX_o2_e': 20.0, 'EX_tungs_e': 0, 'EX_slnt_e': 0})
    model.medium = medium

    # model.add_boundary(model.metabolites.get_by_id('accoa_c'), type = 'demand')
    
    # model.objective = 'AGMHE'
    # model.objective = 'DM_accoa_c'
    # model.objective = 'DM_phb_c'
    model.objective = 'ACACT1r'
    # model.objective = {model.reactions.BIOMASS_Ec_iML1515_core_75p37M: 1, model.reactions.DM_phb_c: 0.02}
    
    # model.reactions.PHAAr.knock_out() # PHAAr

    # model.reactions.GAPD.knock_out() # gapA
    # model.reactions.GAPN.lower_bound = -9.094641
    
    solution = model.optimize()
    # pfba_solution = pfba(model)
    
    print('biomass yield: ', solution.fluxes.get('BIOMASS_Ec_iML1515_core_75p37M'));
    # print('accoa yield: ', solution.fluxes.get('DM_accoa_c'))
    # print('phb yield: ', solution.fluxes.get('DM_phb_c'))
    # print('gapn: ', solution.fluxes.get('GAPN'))
    # print('gapd: ', solution.fluxes.get('GAPD'))
    
    print(model.metabolites.accoa_c.summary());
    # print(model.metabolites.phb_c.summary());
    
    print(model.summary())

    # cobra.io.save_matlab_model(model, "../../models/mat/test.mat")
    

biomass yield:  0.0
accoa_c
Formula: C23H34N7O17P3S

Producing Reactions
-------------------
Percent   Flux Reaction                                                         Definition
 31.44%  7.073    ACALD                 acald_c + coa_c + nad_c <=> accoa_c + h_c + nadh_c
 37.13%  8.353      PDH                 coa_c + nad_c + pyr_c --> accoa_c + co2_c + nadh_c
  0.95% 0.2133      PFL                                  coa_c + pyr_c --> accoa_c + for_c
 30.49%   6.86     POR5 coa_c + 2.0 flxso_c + pyr_c <=> accoa_c + co2_c + 2.0 flxr_c + h_c

Consuming Reactions
-------------------
Percent Flux Reaction                             Definition
 66.67%  -15  ACACT1r        2.0 accoa_c <=> aacoa_c + coa_c
 33.33% -7.5  ACACT2r accoa_c + btcoa_c <=> 3ohcoa_c + coa_c
Objective
1.0 ACACT1r = 7.500000000000015

Uptake
------
Metabolite    Reaction Flux  C-Number  C-Flux
  glc__D_e EX_glc__D_e   10         6 100.00%

Secretion
---------
Metabolite Reaction Flux  C-Number C-Flux
     co2_e EX_co

In [73]:
# fva to test overexpression thresholds
# by knocking out pta and ackA we can see how much acs will be used at baseline
# then just round up to nearest .5 to force overexpression

with model:
    
    medium = model.medium
    # medium.update({'EX_glc__D_e': 1.0, 'EX_o2_e': 20.0})
    medium.update({'EX_glc__D_e': 10.0, 'EX_phleth_e': 0.0, 'EX_o2_e': 20.0})
    # medium.update({'EX_pi_e': 1000.0, 'EX_co2_e': 0, 'EX_fe3_e': 1000.0, 'EX_h_e': 1000.0, 'EX_mn2_e': 1000.0, 'EX_fe2_e': 0, 'EX_glc__D_e': 10.0, 'EX_zn2_e': 1000.0, 'EX_mg2_e': 1000.0, 'EX_ca2_e': 1000.0, 'EX_ni2_e': 0, 'EX_cu2_e': 1000.0, 'EX_sel_e': 0, 'EX_cobalt2_e': 1000.0, 'EX_h2o_e': 1000.0, 'EX_mobd_e': 1000.0, 'EX_so4_e': 1000.0, 'EX_nh4_e': 1000.0, 'EX_k_e': 1000.0, 'EX_na1_e': 1000.0, 'EX_cl_e': 1000.0, 'EX_o2_e': 20.0, 'EX_tungs_e': 0, 'EX_slnt_e': 0})
    model.medium = medium

    model.add_boundary(model.metabolites.get_by_id('accoa_c'), type = 'demand')
    
    # model.objective = 'DM_accoa_c'
    # model.objective = 'DM_phb_c'
    
    # model.reactions.PTAr.knock_out() # pta
    # model.reactions.ACKr.knock_out() # ackA
    
    model.reactions.GAPD.knock_out() # gapA
    
    model.optimize()
    
    fva_rxns = [model.reactions.ACS, model.reactions.GAPD, model.reactions.GAPN]
    print(flux_variability_analysis(model, reaction_list = fva_rxns, fraction_of_optimum = 0.95, loopless = True))
    
    # print(model.summary(fva = 0.95))
    # print(model.metabolites.accoa_c.summary(fva = 0.95))


Set parameter Username
Academic license - for non-commercial use only - expires 2022-03-01
Read LP format model from file /var/folders/9w/hszfvw392q983b9b3bn4zqsw0000gn/T/tmpvq3wgvd0.lp
Reading time = 0.03 seconds
: 1884 rows, 5447 columns, 21229 nonzeros
Set parameter Username
Academic license - for non-commercial use only - expires 2022-03-01
Read LP format model from file /var/folders/9w/hszfvw392q983b9b3bn4zqsw0000gn/T/tmpo6dfuk3b.lp
Reading time = 0.03 seconds
: 1884 rows, 5447 columns, 21229 nonzeros
Set parameter Username
Academic license - for non-commercial use only - expires 2022-03-01
Read LP format model from file /var/folders/9w/hszfvw392q983b9b3bn4zqsw0000gn/T/tmpgh5e67q2.lp
Reading time = 0.03 seconds
: 1884 rows, 5447 columns, 21229 nonzeros
Set parameter Username
Academic license - for non-commercial use only - expires 2022-03-01
Read LP format model from file /var/folders/9w/hszfvw392q983b9b3bn4zqsw0000gn/T/tmp65bhtijs.lp
Reading time = 0.03 seconds
: 1884 rows, 5447 

In [153]:
# pta + ack knockout, acs overexpression

with model:
    
    medium = model.medium
    # medium.update({'EX_glc__D_e': 10.0, 'EX_o2_e': 20.0})
    medium.update({'EX_glc__D_e': 10.0, 'EX_phleth_e': 0.0, 'EX_o2_e': 20.0})
    # medium.update({'EX_pi_e': 1000.0, 'EX_co2_e': 0, 'EX_fe3_e': 1000.0, 'EX_h_e': 1000.0, 'EX_mn2_e': 1000.0, 'EX_fe2_e': 0, 'EX_glc__D_e': 10.0, 'EX_zn2_e': 1000.0, 'EX_mg2_e': 1000.0, 'EX_ca2_e': 1000.0, 'EX_ni2_e': 0, 'EX_cu2_e': 1000.0, 'EX_sel_e': 0, 'EX_cobalt2_e': 1000.0, 'EX_h2o_e': 1000.0, 'EX_mobd_e': 1000.0, 'EX_so4_e': 1000.0, 'EX_nh4_e': 1000.0, 'EX_k_e': 1000.0, 'EX_na1_e': 1000.0, 'EX_cl_e': 1000.0, 'EX_o2_e': 20.0, 'EX_tungs_e': 0, 'EX_slnt_e': 0})
    model.medium = medium
    
    
    # model.add_boundary(model.metabolites.get_by_id('accoa_c'), type = 'demand')
    
    # model.objective = 'DM_accoa_c'
    # model.objective = 'DM_phb_c'
    model.objective = {model.reactions.BIOMASS_Ec_iML1515_core_75p37M: 1, model.reactions.DM_phb_c: 0.02}
    
    with model:
        try:
            # model.reactions.GAPN.knock_out() # gapN
            solution = model.optimize()
        except:
            solution = model.optimize()
    
    print('biomass yield wt: ', solution.fluxes.get('BIOMASS_Ec_iML1515_core_75p37M'))
    print('phb yield wt: ', solution.fluxes.get('DM_phb_c'))
    
    old = float(solution.fluxes.get('DM_phb_c'))
    
    # maintain biomass production
    # model.reactions.BIOMASS_Ec_iML1515_core_75p37M.lower_bound = 0.1
    
    # minimum metabolite yield
    # model.reactions.DM_accoa_c.lower_bound = 0.1
    # model.reactions.DM_phb_c.lower_bound = 1
    
    # try:
        # model.reactions.GAPD.knock_out() # gapA
        # model.reactions.GAPN.knock_out() # gapN
        # model.reactions.GAPN.lower_bound = 9 #-9.094641
    # except:
        # pass
    
    # acetate modifications
    # model.reactions.PTAr.knock_out() # pta
    # model.reactions.ACKr.knock_out() # ackA
    # model.reactions.ACS.lower_bound = 1.5 # acs
    
    # tca modifications
    # model.reactions.SUCOAS.knock_out() # sucC (succinyl coa synthetase)
    model.reactions.SUCDi.knock_out() # sdhCAB (succinate dehydrogenase)
    # model.reactions.FUM.knock_out() # fumC (fumarase)

    # others
    # model.reactions.ACOAD6f.knock_out()
    # model.reactions.POX.knock_out() # POX (pyruvate oxidase)
    # model.reactions.AGMHE.knock_out() # LPS truncation
    # model.reactions.GMHEPPA.knock_out()
    # model.reactions.GMHEPAT.knock_out()
    # model.reactions.GMHEPK.knock_out()
    
    # model.reactions.PYK3.knock_out()
    # model.reactions.IPDDI.knock_out()
    # model.reactions.CITtex.knock_out()
    
    model.reactions.GLUDy.knock_out()
    model.reactions.GLUSy.knock_out()
    model.reactions.NADTRHD.knock_out()
    model.reactions.LCARSyi.knock_out()
    model.reactions.get_by_id('5DGLCNR').knock_out()
    model.reactions.IDOND2.knock_out()
    model.reactions.FLDR2.knock_out()
    model.reactions.HPYRRy.knock_out()
    model.reactions.ATHRDHr.knock_out()
    model.reactions.TRDR.knock_out()
    # model.reactions.GTHOr.knock_out()
    
    # solution = model.optimize()
    # moma_solution = moma(model, solution)
    # print(dir(moma_solution))
    
    # print(model.summary(fva = 0.95))

    # cobra.io.save_matlab_model(model, "../../models/mat/test.mat")
    # cobra.io.save_json_model(model, "../../models/json/test.json")
    
    solution = model.optimize()

    print('biomass yield: ', solution.fluxes.get('BIOMASS_Ec_iML1515_core_75p37M'))
    # print('biomass yield (moma): ', moma_solution.fluxes.get('BIOMASS_Ec_iML1515_core_75p37M'))
    # print('ATP maintenance: ', solution.fluxes.get('ATPM'))
    # print('accoa yield: ', solution.fluxes.get('DM_accoa_c'))
    # print('accoa yield (moma): ', moma_solution.fluxes.get('DM_accoa_c'))
    print('phb yield: ', solution.fluxes.get('DM_phb_c'))
    # print('phb yield (moma): ', moma_solution.fluxes.get('DM_phb_c'))
    
    new = float(solution.fluxes.get('DM_phb_c'))
    dif = new-old
    print('\nnumerical increase phb: ' + str(dif))
    try:
        pdif = (new-old)/old*100
        print('percent increase phb: ' + str(pdif) +'%\n')
    except:
        print('\nfuck\n')
        pass
    
    # print(model.metabolites.accoa_c.summary());
    print(model.metabolites.nadp_c.summary());
    # print(model.metabolites.nadph_c.summary());
    # print(model.metabolites.g3p_c.summary());

    
    print(model.summary())
    # print(model.summary(fva = 0.95))
    
    # shadow = solution.shadow_prices
    # shadow.to_csv('shadow.csv')
    
    # t = sample(model, 20, processes = 2)


biomass yield wt:  0.7986056852720441
phb yield wt:  1.225579029336875
biomass yield:  0.596966506977755
phb yield:  3.160739686442681

numerical increase phb: 1.935160657105806
percent increase phb: 157.89766394361897%

nadp_c
Formula: C21H25N7O17P3

Producing Reactions
-------------------
Percent      Flux Reaction                                                                      Definition
  2.01%    0.2124  3OAR100                              3odecACP_c + h_c + nadph_c <=> 3hdecACP_c + nadp_c
  1.16%    0.1226  3OAR120                            3oddecACP_c + h_c + nadph_c <=> 3hddecACP_c + nadp_c
  0.85%    0.0898  3OAR121                      3ocddec5eACP_c + h_c + nadph_c --> 3hcddec5eACP_c + nadp_c
  1.16%    0.1226  3OAR140                              3omrsACP_c + h_c + nadph_c <=> 3hmrsACP_c + nadp_c
  0.85%    0.0898  3OAR141                        3ocmrs7eACP_c + h_c + nadph_c --> 3hcmrs7eACP_c + nadp_c
  0.72%   0.07619  3OAR160                            3opalmACP_c 

In [270]:
s.to_csv('sampling/s.csv', index = False)
t.to_csv('sampling/t.csv', index = False)